In [1]:
import pandas as pd
import numpy as np

import requests
import json
from datetime import datetime

**Import df_cleaned.csv**

In [2]:
df_cleaned = pd.read_csv('df_cleaned.csv')

In [4]:
df_cleaned.head()

,Unnamed: 0,Record ID,Incident Year,Incident Month,Incident Day,Operator ID,Operator,Aircraft,Aircraft Type,Aircraft Make,...,Flight Phase,Visibility,Precipitation,Height,Speed,Distance,Species ID,Species Name,Species Quantity,Flight Impact
0,106753,300045,2010,1,1,BUS,BUSINESS,C-310,A,226,...,TAKEOFF RUN,DAY,NONE,0.0,30.0,0.0,NE1,GULL,1,ABORTED TAKEOFF
1,106754,300390,2010,1,1,JBU,JETBLUE AIRWAYS,A-320,A,04A,...,TAKEOFF RUN,DAY,RAIN,0.0,120.0,0.0,ZT3,GRACKLE,1,NONE
2,106764,300701,2010,1,2,SKW,SKYWEST AIRLINES,CRJ100/200,A,188,...,LANDING ROLL,DAY,NONE,0.0,80.0,0.0,YH004,HORNED LARK,1,NONE
3,106765,300186,2010,1,2,NWA,NORTHWEST AIRLINES,DC-9,A,583,...,TAKEOFF RUN,DAY,NONE,0.0,140.0,0.0,K33,HAWK,1,NONE
4,106769,301152,2010,1,2,JBU,JETBLUE AIRWAYS,A-320,A,04A,...,APPROACH,DAY,NONE,20.0,120.0,0.0,YI010,TREE SWALLOW,1,NONE


**Create birdstrike count for airports and save as csv**

In [5]:
airport_counts = df_cleaned.groupby('Airport ID').size().sort_values(ascending=False)
df_airports = pd.DataFrame({'Airport ID':airport_counts.index, 'Birdstrikes_N':airport_counts.values})

In [6]:
df_airports.head()

,Airport ID,Birdstrikes_N
0,KDFW,402
1,KSMF,337
2,KDEN,293
3,KPHL,286
4,KBNA,258


**Import airport_codes_csv (downloaded from internet)**

In [7]:
airport_codes = pd.read_csv('airport-codes_csv.csv')

In [8]:
airport_codes.head()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"


In [9]:
# airport_codes[airport_codes['ident'] == 'KJAX']
# name municipality gps_code coordinates

In [10]:
#df_merged = pd.merge(airports, airport_codes, how = 'left', left_on = 'Airport ID', right_on = 'ident')

In [11]:
#df_merged['long'], df_merged['lat'] = df_merged['coordinates'].str.split(',', 1).str

In [13]:
#pd.set_option('display.max_rows', None)

In [14]:
len(airport_codes)

57421

**Merging df_airports and airport_codes to get long/lat values for airport IDs that only our data has**

In [15]:
df_merged = pd.merge(df_airports, airport_codes, how = 'inner', left_on = 'Airport ID', right_on = 'ident')

In [16]:
df_merged['long'], df_merged['lat'] = df_merged['coordinates'].str.split(',', 1).str

<ipython-input-16-e3d884dec4d5>:1: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  df_merged['long'], df_merged['lat'] = df_merged['coordinates'].str.split(',', 1).str


In [17]:
df_merged['lat'] = df_merged['lat'].astype(str)
df_merged['long'] = df_merged['long'].astype(str)

In [18]:
df_merged.head()

,Airport ID,Birdstrikes_N,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates,long,lat
0,KDFW,402,KDFW,large_airport,Dallas Fort Worth International Airport,607.0,NaN,US,US-TX,Dallas-Fort Worth,KDFW,DFW,DFW,"-97.038002, 32.896801",-97.038002,32.896801
1,KSMF,337,KSMF,large_airport,Sacramento International Airport,27.0,NaN,US,US-CA,Sacramento,KSMF,SMF,SMF,"-121.59100341796875, 38.69540023803711",-121.59100341796875,38.69540023803711
2,KDEN,293,KDEN,large_airport,Denver International Airport,5431.0,NaN,US,US-CO,Denver,KDEN,DEN,DEN,"-104.672996521, 39.861698150635",-104.672996521,39.861698150635
3,KPHL,286,KPHL,large_airport,Philadelphia International Airport,36.0,NaN,US,US-PA,Philadelphia,KPHL,PHL,PHL,"-75.24109649658203, 39.87189865112305",-75.24109649658203,39.87189865112305
4,KBNA,258,KBNA,large_airport,Nashville International Airport,599.0,NaN,US,US-TN,Nashville,KBNA,BNA,BNA,"-86.6781997680664, 36.1245002746582",-86.6781997680664,36.1245002746582


**Create empty zips with lat/long df**

In [19]:
column_names = ["lat_", "long_", "zipcode"]
df_w_zip = pd.DataFrame(columns = column_names)

**Import zipcodes via API**

In [20]:
Token = '546a860b4a7a4d0697f2fe7f66f1e7e5'

In [21]:
# initially, list was used to store zip codes, then switched to df
# zip_codes_list = []

In [23]:
for index, row in df_merged.iterrows():
    #make the api call
    response = requests.get('https://api.bigdatacloud.net/data/reverse-geocode?latitude='+row.lat+'&longitude='+row.long+'&localityLanguage=en&key='+Token)  
    #load the api response as a json
    res_json = json.loads(response.text)
    if res_json['postcode'] == None:
        continue
    else:
        new_row = {'lat_':row.lat, 'long_': row.long, 'zipcode': res_json['postcode']}
        # print(new_row)
        df_w_zip = df_w_zip.append(new_row, ignore_index=True)

In [24]:
len(df_w_zip)

1353

In [26]:
df_w_zip.head()

,lat_,long_,zipcode
0,32.896801,-97.038002,76051
1,38.69540023803711,-121.59100341796875,95837
2,39.861698150635,-104.672996521,80249
3,39.87189865112305,-75.24109649658203,19113
4,36.1245002746582,-86.6781997680664,37214


In [31]:
#df_w_zip.to_csv('df_w_zip.csv', index = False)
pd.read_csv('df_w_zip.csv')  # Save time instead of running API again

,lat_,long_,zipcode
0,32.896801,-97.038002,76051.0
1,38.695400,-121.591003,95837.0
2,39.861698,-104.672997,80249.0
3,39.871899,-75.241096,19113.0
4,36.124500,-86.678200,37214.0
...,...,...,...
1348,31.084900,-83.803299,31788.0
1349,39.140999,-96.670799,66503.0
1350,35.599899,-88.915604,38301.0
1351,34.882401,-95.783501,74501.0


**Join all dates together into one column**

In [32]:
df_cleaned["incident_date"] = df_cleaned["Incident Year"].astype(str) + "-" + df_cleaned["Incident Month"].astype(str) + "-" + df_cleaned["Incident Day"].astype(str)
df_cleaned["incident_date"] = pd.to_datetime(df_cleaned['incident_date'], format= '%Y-%m-%d').astype(str)
# date_list = pd.DataFrame(df_cleaned['incident_date'])
# date_list['incident_date'] = date_list['incident_date'].astype(str)

In [33]:
len(df_cleaned)

11675

In [34]:
#len(df_cleaned[df_cleaned['incident_date'].str.contains("2010")])

In [35]:
df_cleaned_coord = pd.merge(df_cleaned, df_merged, how = 'left', left_on = 'Airport ID', right_on = 'Airport ID')

In [36]:
df_cleaned_coord.head()

,Unnamed: 0,Record ID,Incident Year,Incident Month,Incident Day,Operator ID,Operator,Aircraft,Aircraft Type,Aircraft Make,...,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates,long,lat
0,106753,300045,2010,1,1,BUS,BUSINESS,C-310,A,226,...,NaN,US,US-FL,Pompano Beach,KPMP,PPM,PMP,"-80.111099243164, 26.247100830078",-80.111099243164,26.247100830078
1,106754,300390,2010,1,1,JBU,JETBLUE AIRWAYS,A-320,A,04A,...,NaN,US,US-FL,Orlando,KMCO,MCO,MCO,"-81.30899810791016, 28.429399490356445",-81.30899810791016,28.429399490356445
2,106764,300701,2010,1,2,SKW,SKYWEST AIRLINES,CRJ100/200,A,188,...,NaN,US,US-UT,Salt Lake City,KSLC,SLC,SLC,"-111.97799682617188, 40.78839874267578",-111.97799682617188,40.78839874267578
3,106765,300186,2010,1,2,NWA,NORTHWEST AIRLINES,DC-9,A,583,...,NaN,US,US-MD,Baltimore,KBWI,BWI,BWI,"-76.668297, 39.1754",-76.668297,39.1754
4,106769,301152,2010,1,2,JBU,JETBLUE AIRWAYS,A-320,A,04A,...,NaN,US,US-FL,Orlando,KMCO,MCO,MCO,"-81.30899810791016, 28.429399490356445",-81.30899810791016,28.429399490356445


In [37]:
df_cleaned_coord = df_cleaned_coord.drop(columns=['continent','iso_country','iso_region','gps_code','iata_code','local_code'])

In [38]:
df_cleaned_coord.columns

Index(['Unnamed: 0', 'Record ID', 'Incident Year', 'Incident Month',
       'Incident Day', 'Operator ID', 'Operator', 'Aircraft', 'Aircraft Type',
       'Aircraft Make', 'Aircraft Model', 'Airport ID', 'Airport', 'State',
       'FAA Region', 'Flight Phase', 'Visibility', 'Precipitation', 'Height',
       'Speed', 'Distance', 'Species ID', 'Species Name', 'Species Quantity',
       'Flight Impact', 'incident_date', 'Birdstrikes_N', 'ident', 'type',
       'name', 'elevation_ft', 'municipality', 'coordinates', 'long', 'lat'],
      dtype='object')

In [39]:
df_zip_date = pd.merge(df_cleaned_coord, df_w_zip, how = 'left', left_on = ['long', 'lat'], right_on = ['long_', 'lat_'])

In [40]:
df_zip_date.head()

,Unnamed: 0,Record ID,Incident Year,Incident Month,Incident Day,Operator ID,Operator,Aircraft,Aircraft Type,Aircraft Make,...,type,name,elevation_ft,municipality,coordinates,long,lat,lat_,long_,zipcode
0,106753,300045,2010,1,1,BUS,BUSINESS,C-310,A,226,...,small_airport,Pompano Beach Airpark,19.0,Pompano Beach,"-80.111099243164, 26.247100830078",-80.111099243164,26.247100830078,26.247100830078,-80.111099243164,33060
1,106753,300045,2010,1,1,BUS,BUSINESS,C-310,A,226,...,small_airport,Pompano Beach Airpark,19.0,Pompano Beach,"-80.111099243164, 26.247100830078",-80.111099243164,26.247100830078,26.247100830078,-80.111099243164,33060
2,106754,300390,2010,1,1,JBU,JETBLUE AIRWAYS,A-320,A,04A,...,large_airport,Orlando International Airport,96.0,Orlando,"-81.30899810791016, 28.429399490356445",-81.30899810791016,28.429399490356445,28.429399490356445,-81.30899810791016,32827
3,106754,300390,2010,1,1,JBU,JETBLUE AIRWAYS,A-320,A,04A,...,large_airport,Orlando International Airport,96.0,Orlando,"-81.30899810791016, 28.429399490356445",-81.30899810791016,28.429399490356445,28.429399490356445,-81.30899810791016,32827
4,106764,300701,2010,1,2,SKW,SKYWEST AIRLINES,CRJ100/200,A,188,...,large_airport,Salt Lake City International Airport,4227.0,Salt Lake City,"-111.97799682617188, 40.78839874267578",-111.97799682617188,40.78839874267578,40.78839874267578,-111.97799682617188,84116


In [41]:
df_zip_date['zipcode'].isna().sum()

73

In [42]:
df_zip_date = df_zip_date[df_zip_date["zipcode"] != ""]

In [43]:
len(df_zip_date)

22873

In [44]:
df_zip_date.columns

Index(['Unnamed: 0', 'Record ID', 'Incident Year', 'Incident Month',
       'Incident Day', 'Operator ID', 'Operator', 'Aircraft', 'Aircraft Type',
       'Aircraft Make', 'Aircraft Model', 'Airport ID', 'Airport', 'State',
       'FAA Region', 'Flight Phase', 'Visibility', 'Precipitation', 'Height',
       'Speed', 'Distance', 'Species ID', 'Species Name', 'Species Quantity',
       'Flight Impact', 'incident_date', 'Birdstrikes_N', 'ident', 'type',
       'name', 'elevation_ft', 'municipality', 'coordinates', 'long', 'lat',
       'lat_', 'long_', 'zipcode'],
      dtype='object')

In [80]:
df_zip_date_2010 = df_zip_date[df_zip_date['incident_date'].str.contains("2010")]
df_zip_date_2011 = df_zip_date[df_zip_date['incident_date'].str.contains("2011")]

In [81]:
df_zip_date_2010['zipcode'] = df_zip_date_2010['zipcode'].astype(str)
df_zip_date_2011['zipcode'] = df_zip_date_2011['zipcode'].astype(str)

<ipython-input-81-9700c8161968>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_zip_date_2010['zipcode'] = df_zip_date_2010['zipcode'].astype(str)
<ipython-input-81-9700c8161968>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_zip_date_2011['zipcode'] = df_zip_date_2011['zipcode'].astype(str)


In [47]:
# Abduvosids
Token_w = 'EHZroSHTsmfKuOMeOWRkrDpMPWoZTXmb' 

#Fasihs
Token_f = 'KHliZgSorqkyvUBivnqDQPLShGaEkNui'

From the documentation:

ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt

https://stackoverflow.com/questions/49280302/noaa-api-returning-impossible-historical-temperature-values

TOBS = Temperature at the time of observation (tenths of degrees C). We can divide results by 10

Documentation for GCHND dataset: Other measurements we can look at
https://www.ncei.noaa.gov/data/global-historical-climatology-network-daily/doc/GHCND_documentation.pdf


Temperature data not imported due to constant errors by tht etime list gets to July.

In [48]:
#df_w_weather.to_csv('df_w_weather.csv', index=False)

**Function to import weather data**

In [ ]:
def get_weather(datatypeid):
    token = {'token': 'KHliZgSorqkyvUBivnqDQPLShGaEkNui'}
    column_names = ["date", "station", "zipcode", "value"]
    df = pd.DataFrame(columns = column_names)
    
    for index,row in df_zip_date_2010.iterrows():
        response = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&locationid=ZIP:'+row.zipcode+'&datatypeid='+datatypeid+'&startdate='+row.incident_date+'&enddate='+row.incident_date, 
                             headers=token)
        res_json = json.loads(response.text)
        if res_json != {}:
            new_row = {'date': row.incident_date, 'station': res_json['results'][0]['station'], 'zipcode': row.zipcode, 'value': res_json['results'][0]['value']}
            df = df.append(new_row, ignore_index=True)
            print(new_row)
    return(df)

**Import Precipitation data**

In [92]:
df_w_prcp = get_weather("PRCP")

{'date': '2010-01-01', 'station': 'GHCND:USW00092805', 'zipcode': '33060', 'value': 23}
{'date': '2010-01-01', 'station': 'GHCND:USW00092805', 'zipcode': '33060', 'value': 23}
{'date': '2010-01-01', 'station': 'GHCND:USW00012815', 'zipcode': '32827', 'value': 150}
{'date': '2010-01-01', 'station': 'GHCND:USW00012815', 'zipcode': '32827', 'value': 150}
{'date': '2010-01-02', 'station': 'GHCND:USW00024127', 'zipcode': '84116', 'value': 10}
{'date': '2010-01-02', 'station': 'GHCND:USW00024127', 'zipcode': '84116', 'value': 10}
{'date': '2010-01-02', 'station': 'GHCND:USW00093721', 'zipcode': '21240', 'value': 0}
{'date': '2010-01-02', 'station': 'GHCND:USW00093721', 'zipcode': '21240', 'value': 0}
{'date': '2010-01-02', 'station': 'GHCND:USW00012815', 'zipcode': '32827', 'value': 0}
{'date': '2010-01-02', 'station': 'GHCND:USW00012815', 'zipcode': '32827', 'value': 0}
{'date': '2010-01-02', 'station': 'GHCND:USW00024229', 'zipcode': '97218', 'value': 0}
{'date': '2010-01-02', 'station': '

{'date': '2010-01-17', 'station': 'GHCND:USW00013722', 'zipcode': '27560', 'value': 371}
{'date': '2010-01-17', 'station': 'GHCND:USW00013722', 'zipcode': '27560', 'value': 371}
{'date': '2010-01-17', 'station': 'GHCND:USC00511038', 'zipcode': '96766', 'value': 0}
{'date': '2010-01-17', 'station': 'GHCND:USC00511038', 'zipcode': '96766', 'value': 0}
{'date': '2010-01-18', 'station': 'GHCND:USC00511065', 'zipcode': '96720', 'value': 0}
{'date': '2010-01-18', 'station': 'GHCND:USC00511065', 'zipcode': '96720', 'value': 0}
{'date': '2010-01-19', 'station': 'GHCND:USW00012815', 'zipcode': '32827', 'value': 0}
{'date': '2010-01-19', 'station': 'GHCND:USW00012815', 'zipcode': '32827', 'value': 0}
{'date': '2010-01-19', 'station': 'GHCND:USC00500275', 'zipcode': '99502', 'value': 0}
{'date': '2010-01-19', 'station': 'GHCND:USC00500275', 'zipcode': '99502', 'value': 0}
{'date': '2010-01-19', 'station': 'GHCND:US1MTMS0009', 'zipcode': '59808', 'value': 3}
{'date': '2010-01-19', 'station': 'GHCN

{'date': '2010-02-13', 'station': 'GHCND:USW00093225', 'zipcode': '95837', 'value': 3}
{'date': '2010-02-13', 'station': 'GHCND:USW00093225', 'zipcode': '95837', 'value': 3}
{'date': '2010-02-15', 'station': 'GHCND:USW00012815', 'zipcode': '32827', 'value': 0}
{'date': '2010-02-15', 'station': 'GHCND:USW00012815', 'zipcode': '32827', 'value': 0}
{'date': '2010-02-15', 'station': 'GHCND:USC00083163', 'zipcode': '33315', 'value': 0}
{'date': '2010-02-15', 'station': 'GHCND:USC00083163', 'zipcode': '33315', 'value': 0}
{'date': '2010-02-16', 'station': 'GHCND:USC00041194', 'zipcode': '91505', 'value': 0}
{'date': '2010-02-16', 'station': 'GHCND:USC00041194', 'zipcode': '91505', 'value': 0}
{'date': '2010-02-16', 'station': 'GHCND:USW00013881', 'zipcode': '28208', 'value': 0}
{'date': '2010-02-16', 'station': 'GHCND:USW00013881', 'zipcode': '28208', 'value': 0}
{'date': '2010-02-17', 'station': 'GHCND:USW00093225', 'zipcode': '95837', 'value': 3}
{'date': '2010-02-17', 'station': 'GHCND:US

{'date': '2010-03-12', 'station': 'GHCND:USW00012815', 'zipcode': '32827', 'value': 305}
{'date': '2010-03-12', 'station': 'GHCND:USW00012815', 'zipcode': '32827', 'value': 305}
{'date': '2010-03-14', 'station': 'GHCND:USW00023129', 'zipcode': '90807', 'value': 0}
{'date': '2010-03-14', 'station': 'GHCND:USW00023129', 'zipcode': '90807', 'value': 0}
{'date': '2010-03-15', 'station': 'GHCND:USW00023169', 'zipcode': '89119', 'value': 0}
{'date': '2010-03-15', 'station': 'GHCND:USW00023169', 'zipcode': '89119', 'value': 0}
{'date': '2010-03-15', 'station': 'GHCND:USW00003947', 'zipcode': '64153', 'value': 0}
{'date': '2010-03-15', 'station': 'GHCND:USW00003947', 'zipcode': '64153', 'value': 0}
{'date': '2010-03-15', 'station': 'GHCND:USW00023234', 'zipcode': '94128', 'value': 0}
{'date': '2010-03-15', 'station': 'GHCND:USW00023234', 'zipcode': '94128', 'value': 0}
{'date': '2010-03-15', 'station': 'GHCND:USW00093225', 'zipcode': '95837', 'value': 0}
{'date': '2010-03-15', 'station': 'GHCN

{'date': '2010-04-03', 'station': 'GHCND:USW00024233', 'zipcode': '98158', 'value': 43}
{'date': '2010-04-03', 'station': 'GHCND:USW00024233', 'zipcode': '98158', 'value': 43}
{'date': '2010-04-03', 'station': 'GHCND:USW00014740', 'zipcode': '06096', 'value': 0}
{'date': '2010-04-03', 'station': 'GHCND:USW00014740', 'zipcode': '06096', 'value': 0}
{'date': '2010-04-03', 'station': 'GHCND:USW00023183', 'zipcode': '85034', 'value': 0}
{'date': '2010-04-03', 'station': 'GHCND:USW00023183', 'zipcode': '85034', 'value': 0}
{'date': '2010-04-03', 'station': 'GHCND:USC00047298', 'zipcode': '96002', 'value': 0}
{'date': '2010-04-03', 'station': 'GHCND:USC00047298', 'zipcode': '96002', 'value': 0}
{'date': '2010-04-04', 'station': 'GHCND:USW00023174', 'zipcode': '90045', 'value': 3}
{'date': '2010-04-04', 'station': 'GHCND:USW00023174', 'zipcode': '90045', 'value': 3}
{'date': '2010-04-04', 'station': 'GHCND:USW00024229', 'zipcode': '97218', 'value': 20}
{'date': '2010-04-04', 'station': 'GHCND

...

{'date': '2010-10-21', 'station': 'GHCND:USC00516395', 'zipcode': '96818', 'value': 48}
{'date': '2010-10-21', 'station': 'GHCND:USW00023188', 'zipcode': '92101', 'value': 0}
{'date': '2010-10-21', 'station': 'GHCND:USW00023188', 'zipcode': '92101', 'value': 0}
{'date': '2010-10-21', 'station': 'GHCND:USW00023160', 'zipcode': '85756', 'value': 18}
{'date': '2010-10-21', 'station': 'GHCND:USW00023160', 'zipcode': '85756', 'value': 18}
{'date': '2010-10-21', 'station': 'GHCND:USW00003017', 'zipcode': '80249', 'value': 0}
{'date': '2010-10-21', 'station': 'GHCND:USW00003017', 'zipcode': '80249', 'value': 0}
{'date': '2010-10-21', 'station': 'GHCND:USC00111577', 'zipcode': '60638', 'value': 0}
{'date': '2010-10-21', 'station': 'GHCND:USC00111577', 'zipcode': '60638', 'value': 0}
{'date': '2010-10-21', 'station': 'GHCND:USW00014740', 'zipcode': '06096', 'value': 10}
{'date': '2010-10-21', 'station': 'GHCND:USW00014740', 'zipcode': '06096', 'value': 10}
{'date': '2010-10-21', 'station': 'GHC

{'date': '2010-10-29', 'station': 'GHCND:US1TXTN0023', 'zipcode': '76051', 'value': 0}
{'date': '2010-10-29', 'station': 'GHCND:USW00012916', 'zipcode': '70062', 'value': 0}
{'date': '2010-10-29', 'station': 'GHCND:USW00012916', 'zipcode': '70062', 'value': 0}
{'date': '2010-10-29', 'station': 'GHCND:USW00012815', 'zipcode': '32827', 'value': 0}
{'date': '2010-10-29', 'station': 'GHCND:USW00012815', 'zipcode': '32827', 'value': 0}
{'date': '2010-10-29', 'station': 'GHCND:USW00024229', 'zipcode': '97218', 'value': 5}
{'date': '2010-10-29', 'station': 'GHCND:USW00024229', 'zipcode': '97218', 'value': 5}
{'date': '2010-10-30', 'station': 'GHCND:USC00083163', 'zipcode': '33315', 'value': 0}
{'date': '2010-10-30', 'station': 'GHCND:USC00083163', 'zipcode': '33315', 'value': 0}
{'date': '2010-10-30', 'station': 'GHCND:USC00445204', 'zipcode': '20110', 'value': 0}
{'date': '2010-10-30', 'station': 'GHCND:USC00445204', 'zipcode': '20110', 'value': 0}
{'date': '2010-10-31', 'station': 'GHCND:US

{'date': '2010-11-06', 'station': 'GHCND:USW00094789', 'zipcode': '11430', 'value': 0}
{'date': '2010-11-07', 'station': 'GHCND:USW00093225', 'zipcode': '95837', 'value': 142}
{'date': '2010-11-07', 'station': 'GHCND:USW00093225', 'zipcode': '95837', 'value': 142}
{'date': '2010-11-08', 'station': 'GHCND:USW00014942', 'zipcode': '68110', 'value': 0}
{'date': '2010-11-08', 'station': 'GHCND:USW00014942', 'zipcode': '68110', 'value': 0}
{'date': '2010-11-08', 'station': 'GHCND:USW00023174', 'zipcode': '90045', 'value': 79}
{'date': '2010-11-08', 'station': 'GHCND:USW00023174', 'zipcode': '90045', 'value': 79}
{'date': '2010-11-08', 'station': 'GHCND:USC00346659', 'zipcode': '73159', 'value': 0}
{'date': '2010-11-08', 'station': 'GHCND:USC00346659', 'zipcode': '73159', 'value': 0}
{'date': '2010-11-08', 'station': 'GHCND:USW00023293', 'zipcode': '95110', 'value': 0}
{'date': '2010-11-08', 'station': 'GHCND:USW00023293', 'zipcode': '95110', 'value': 0}
{'date': '2010-11-09', 'station': 'GH

{'date': '2010-11-18', 'station': 'GHCND:USW00094846', 'zipcode': '60018', 'value': 0}
{'date': '2010-11-18', 'station': 'GHCND:USW00094846', 'zipcode': '60018', 'value': 0}
{'date': '2010-11-18', 'station': 'GHCND:USW00023234', 'zipcode': '94128', 'value': 0}
{'date': '2010-11-18', 'station': 'GHCND:USW00023234', 'zipcode': '94128', 'value': 0}
{'date': '2010-11-19', 'station': 'GHCND:USW00093909', 'zipcode': '66031', 'value': 0}
{'date': '2010-11-19', 'station': 'GHCND:USW00093909', 'zipcode': '66031', 'value': 0}
{'date': '2010-11-19', 'station': 'GHCND:USW00014922', 'zipcode': '55450', 'value': 0}
{'date': '2010-11-19', 'station': 'GHCND:USW00014922', 'zipcode': '55450', 'value': 0}
{'date': '2010-11-19', 'station': 'GHCND:US1IAWB0003', 'zipcode': '50501', 'value': 0}
{'date': '2010-11-19', 'station': 'GHCND:US1IAWB0003', 'zipcode': '50501', 'value': 0}
{'date': '2010-11-20', 'station': 'GHCND:USW00094261', 'zipcode': '97124', 'value': 43}
{'date': '2010-11-20', 'station': 'GHCND:U

{'date': '2010-12-08', 'station': 'GHCND:USC00083163', 'zipcode': '33315', 'value': 0}
{'date': '2010-12-08', 'station': 'GHCND:USC00083163', 'zipcode': '33315', 'value': 0}
{'date': '2010-12-09', 'station': 'GHCND:USW00003102', 'zipcode': '91761', 'value': 0}
{'date': '2010-12-09', 'station': 'GHCND:USW00003102', 'zipcode': '91761', 'value': 0}
{'date': '2010-12-11', 'station': 'GHCND:USC00083163', 'zipcode': '33315', 'value': 0}
{'date': '2010-12-11', 'station': 'GHCND:USC00083163', 'zipcode': '33315', 'value': 0}
{'date': '2010-12-11', 'station': 'GHCND:USW00003017', 'zipcode': '80249', 'value': 0}
{'date': '2010-12-11', 'station': 'GHCND:USW00003017', 'zipcode': '80249', 'value': 0}
{'date': '2010-12-11', 'station': 'GHCND:USC00041194', 'zipcode': '91505', 'value': 0}
{'date': '2010-12-11', 'station': 'GHCND:USC00041194', 'zipcode': '91505', 'value': 0}
{'date': '2010-12-11', 'station': 'GHCND:USW00012895', 'zipcode': '34946', 'value': 0}
{'date': '2010-12-11', 'station': 'GHCND:US

In [95]:
df_w_prcp.to_csv('df_w_prcp.csv')

**Import Average Wind Speed data**

In [94]:
df_w_awnd = get_weather("AWND")

{'date': '2010-01-01', 'station': 'GHCND:USW00092805', 'zipcode': '33060', 'value': 41}
{'date': '2010-01-01', 'station': 'GHCND:USW00092805', 'zipcode': '33060', 'value': 41}
{'date': '2010-01-01', 'station': 'GHCND:USW00012815', 'zipcode': '32827', 'value': 38}
{'date': '2010-01-01', 'station': 'GHCND:USW00012815', 'zipcode': '32827', 'value': 38}
{'date': '2010-01-02', 'station': 'GHCND:USW00024127', 'zipcode': '84116', 'value': 16}
{'date': '2010-01-02', 'station': 'GHCND:USW00024127', 'zipcode': '84116', 'value': 16}
{'date': '2010-01-02', 'station': 'GHCND:USW00093721', 'zipcode': '21240', 'value': 96}
{'date': '2010-01-02', 'station': 'GHCND:USW00093721', 'zipcode': '21240', 'value': 96}
{'date': '2010-01-02', 'station': 'GHCND:USW00012815', 'zipcode': '32827', 'value': 48}
{'date': '2010-01-02', 'station': 'GHCND:USW00012815', 'zipcode': '32827', 'value': 48}
{'date': '2010-01-02', 'station': 'GHCND:USW00024229', 'zipcode': '97218', 'value': 37}
{'date': '2010-01-02', 'station'

{'date': '2010-01-19', 'station': 'GHCND:USW00012815', 'zipcode': '32827', 'value': 12}
{'date': '2010-01-19', 'station': 'GHCND:USW00012815', 'zipcode': '32827', 'value': 12}
{'date': '2010-01-19', 'station': 'GHCND:USW00026451', 'zipcode': '99502', 'value': 16}
{'date': '2010-01-19', 'station': 'GHCND:USW00026451', 'zipcode': '99502', 'value': 16}
{'date': '2010-01-19', 'station': 'GHCND:USW00024153', 'zipcode': '59808', 'value': 12}
{'date': '2010-01-19', 'station': 'GHCND:USW00024153', 'zipcode': '59808', 'value': 12}
{'date': '2010-01-19', 'station': 'GHCND:USW00022536', 'zipcode': '96766', 'value': 47}
{'date': '2010-01-19', 'station': 'GHCND:USW00022536', 'zipcode': '96766', 'value': 47}
{'date': '2010-01-20', 'station': 'GHCND:USW00014765', 'zipcode': '02886', 'value': 34}
{'date': '2010-01-20', 'station': 'GHCND:USW00014765', 'zipcode': '02886', 'value': 34}
{'date': '2010-01-20', 'station': 'GHCND:USW00023183', 'zipcode': '85034', 'value': 31}
{'date': '2010-01-20', 'station'

{'date': '2010-02-18', 'station': 'GHCND:USW00093225', 'zipcode': '95837', 'value': 13}
{'date': '2010-02-18', 'station': 'GHCND:USW00093225', 'zipcode': '95837', 'value': 13}
{'date': '2010-02-18', 'station': 'GHCND:USW00014737', 'zipcode': '18109', 'value': 59}
{'date': '2010-02-18', 'station': 'GHCND:USW00014737', 'zipcode': '18109', 'value': 59}
{'date': '2010-02-20', 'station': 'GHCND:USW00003919', 'zipcode': '67401', 'value': 42}
{'date': '2010-02-21', 'station': 'GHCND:USW00012894', 'zipcode': '33913', 'value': 26}
{'date': '2010-02-21', 'station': 'GHCND:USW00012894', 'zipcode': '33913', 'value': 26}
{'date': '2010-02-21', 'station': 'GHCND:USW00093721', 'zipcode': '21240', 'value': 30}
{'date': '2010-02-21', 'station': 'GHCND:USW00093721', 'zipcode': '21240', 'value': 30}
{'date': '2010-02-22', 'station': 'GHCND:USW00093225', 'zipcode': '95837', 'value': 37}
{'date': '2010-02-22', 'station': 'GHCND:USW00093225', 'zipcode': '95837', 'value': 37}
{'date': '2010-02-23', 'station'

{'date': '2010-03-18', 'station': 'GHCND:USW00023234', 'zipcode': '94128', 'value': 26}
{'date': '2010-03-19', 'station': 'GHCND:USW00013874', 'zipcode': '30337', 'value': 28}
{'date': '2010-03-19', 'station': 'GHCND:USW00013874', 'zipcode': '30337', 'value': 28}
{'date': '2010-03-19', 'station': 'GHCND:USW00093225', 'zipcode': '95837', 'value': 34}
{'date': '2010-03-19', 'station': 'GHCND:USW00093225', 'zipcode': '95837', 'value': 34}
{'date': '2010-03-19', 'station': 'GHCND:USW00013876', 'zipcode': '35212', 'value': 5}
{'date': '2010-03-19', 'station': 'GHCND:USW00013876', 'zipcode': '35212', 'value': 5}
{'date': '2010-03-19', 'station': 'GHCND:USW00013874', 'zipcode': '30337', 'value': 28}
{'date': '2010-03-19', 'station': 'GHCND:USW00013874', 'zipcode': '30337', 'value': 28}
{'date': '2010-03-20', 'station': 'GHCND:USW00014710', 'zipcode': '03103', 'value': 26}
{'date': '2010-03-20', 'station': 'GHCND:USW00014710', 'zipcode': '03103', 'value': 26}
{'date': '2010-03-20', 'station': 

{'date': '2010-04-09', 'station': 'GHCND:USW00003102', 'zipcode': '91761', 'value': 26}
{'date': '2010-04-09', 'station': 'GHCND:USW00025333', 'zipcode': '99835', 'value': 17}
{'date': '2010-04-09', 'station': 'GHCND:USW00025333', 'zipcode': '99835', 'value': 17}
{'date': '2010-04-09', 'station': 'GHCND:USW00003816', 'zipcode': '42086', 'value': 20}
{'date': '2010-04-09', 'station': 'GHCND:USW00003816', 'zipcode': '42086', 'value': 20}
{'date': '2010-04-09', 'station': 'GHCND:USW00014707', 'zipcode': '06340', 'value': 45}
{'date': '2010-04-09', 'station': 'GHCND:USW00014707', 'zipcode': '06340', 'value': 45}
{'date': '2010-04-10', 'station': 'GHCND:USW00094846', 'zipcode': '60018', 'value': 51}
{'date': '2010-04-10', 'station': 'GHCND:USW00094846', 'zipcode': '60018', 'value': 51}
{'date': '2010-04-10', 'station': 'GHCND:USW00093819', 'zipcode': '46241', 'value': 38}
{'date': '2010-04-10', 'station': 'GHCND:USW00093819', 'zipcode': '46241', 'value': 38}
{'date': '2010-04-10', 'station'

...

{'date': '2010-10-16', 'station': 'GHCND:USW00013748', 'zipcode': '28405', 'value': 26}
{'date': '2010-10-16', 'station': 'GHCND:USW00012815', 'zipcode': '32827', 'value': 27}
{'date': '2010-10-16', 'station': 'GHCND:USW00012815', 'zipcode': '32827', 'value': 27}
{'date': '2010-10-16', 'station': 'GHCND:USW00003017', 'zipcode': '80249', 'value': 35}
{'date': '2010-10-16', 'station': 'GHCND:USW00003017', 'zipcode': '80249', 'value': 35}
{'date': '2010-10-17', 'station': 'GHCND:USW00024229', 'zipcode': '97218', 'value': 20}
{'date': '2010-10-17', 'station': 'GHCND:USW00024229', 'zipcode': '97218', 'value': 20}
{'date': '2010-10-17', 'station': 'GHCND:USW00023293', 'zipcode': '95110', 'value': 15}
{'date': '2010-10-17', 'station': 'GHCND:USW00023293', 'zipcode': '95110', 'value': 15}
{'date': '2010-10-17', 'station': 'GHCND:USW00014732', 'zipcode': '11371', 'value': 40}
{'date': '2010-10-17', 'station': 'GHCND:USW00014732', 'zipcode': '11371', 'value': 40}
{'date': '2010-10-18', 'station'

{'date': '2010-10-28', 'station': 'GHCND:USW00025501', 'zipcode': '99615', 'value': 41}
{'date': '2010-10-28', 'station': 'GHCND:USW00014734', 'zipcode': '07114', 'value': 51}
{'date': '2010-10-28', 'station': 'GHCND:USW00014734', 'zipcode': '07114', 'value': 51}
{'date': '2010-10-28', 'station': 'GHCND:USW00013899', 'zipcode': '32504', 'value': 43}
{'date': '2010-10-28', 'station': 'GHCND:USW00013899', 'zipcode': '32504', 'value': 43}
{'date': '2010-10-29', 'station': 'GHCND:USW00013893', 'zipcode': '38116', 'value': 19}
{'date': '2010-10-29', 'station': 'GHCND:USW00013893', 'zipcode': '38116', 'value': 19}
{'date': '2010-10-29', 'station': 'GHCND:USW00013893', 'zipcode': '38116', 'value': 19}
{'date': '2010-10-29', 'station': 'GHCND:USW00013893', 'zipcode': '38116', 'value': 19}
{'date': '2010-10-29', 'station': 'GHCND:USW00014765', 'zipcode': '02886', 'value': 40}
{'date': '2010-10-29', 'station': 'GHCND:USW00014765', 'zipcode': '02886', 'value': 40}
{'date': '2010-10-29', 'station'

{'date': '2010-11-05', 'station': 'GHCND:USW00012849', 'zipcode': '33315', 'value': 57}
{'date': '2010-11-06', 'station': 'GHCND:USW00023169', 'zipcode': '89119', 'value': 36}
{'date': '2010-11-06', 'station': 'GHCND:USW00023169', 'zipcode': '89119', 'value': 36}
{'date': '2010-11-06', 'station': 'GHCND:USW00003017', 'zipcode': '80249', 'value': 48}
{'date': '2010-11-06', 'station': 'GHCND:USW00003017', 'zipcode': '80249', 'value': 48}
{'date': '2010-11-06', 'station': 'GHCND:USW00013881', 'zipcode': '28208', 'value': 31}
{'date': '2010-11-06', 'station': 'GHCND:USW00013881', 'zipcode': '28208', 'value': 31}
{'date': '2010-11-06', 'station': 'GHCND:USW00014739', 'zipcode': '02128', 'value': 43}
{'date': '2010-11-06', 'station': 'GHCND:USW00014739', 'zipcode': '02128', 'value': 43}
{'date': '2010-11-06', 'station': 'GHCND:USW00094789', 'zipcode': '11430', 'value': 54}
{'date': '2010-11-06', 'station': 'GHCND:USW00094789', 'zipcode': '11430', 'value': 54}
{'date': '2010-11-07', 'station'

{'date': '2010-11-17', 'station': 'GHCND:USW00013960', 'zipcode': '75235', 'value': 68}
{'date': '2010-11-17', 'station': 'GHCND:USW00013960', 'zipcode': '75235', 'value': 68}
{'date': '2010-11-18', 'station': 'GHCND:USW00094846', 'zipcode': '60018', 'value': 30}
{'date': '2010-11-18', 'station': 'GHCND:USW00094846', 'zipcode': '60018', 'value': 30}
{'date': '2010-11-18', 'station': 'GHCND:USW00023234', 'zipcode': '94128', 'value': 41}
{'date': '2010-11-18', 'station': 'GHCND:USW00023234', 'zipcode': '94128', 'value': 41}
{'date': '2010-11-19', 'station': 'GHCND:USW00093909', 'zipcode': '66031', 'value': 50}
{'date': '2010-11-19', 'station': 'GHCND:USW00093909', 'zipcode': '66031', 'value': 50}
{'date': '2010-11-19', 'station': 'GHCND:USW00014922', 'zipcode': '55450', 'value': 54}
{'date': '2010-11-19', 'station': 'GHCND:USW00014922', 'zipcode': '55450', 'value': 54}
{'date': '2010-11-20', 'station': 'GHCND:USW00094261', 'zipcode': '97124', 'value': 15}
{'date': '2010-11-20', 'station'

{'date': '2010-12-11', 'station': 'GHCND:USW00023152', 'zipcode': '91505', 'value': 14}
{'date': '2010-12-11', 'station': 'GHCND:USW00023152', 'zipcode': '91505', 'value': 14}
{'date': '2010-12-11', 'station': 'GHCND:USW00012895', 'zipcode': '34946', 'value': 16}
{'date': '2010-12-11', 'station': 'GHCND:USW00012895', 'zipcode': '34946', 'value': 16}
{'date': '2010-12-13', 'station': 'GHCND:USW00093741', 'zipcode': '23602', 'value': 56}
{'date': '2010-12-13', 'station': 'GHCND:USW00093741', 'zipcode': '23602', 'value': 56}
{'date': '2010-12-13', 'station': 'GHCND:USW00012815', 'zipcode': '32827', 'value': 78}
{'date': '2010-12-13', 'station': 'GHCND:USW00012815', 'zipcode': '32827', 'value': 78}
{'date': '2010-12-14', 'station': 'GHCND:USW00013904', 'zipcode': '78719', 'value': 48}
{'date': '2010-12-14', 'station': 'GHCND:USW00013904', 'zipcode': '78719', 'value': 48}
{'date': '2010-12-14', 'station': 'GHCND:USW00012894', 'zipcode': '33913', 'value': 31}
{'date': '2010-12-14', 'station'

In [96]:
df_w_awnd.to_csv('df_w_awnd.csv')